In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from google.colab import drive, files
import os


In [ ]:
drive.mount('/content/drive')
dir_path = "/content/drive/MyDrive/DR/colored_images/"


In [ ]:
# import zipfile

# zip_path = "/content/drive/MyDrive/DR/archive.zip"
# extract_path = "/content/drive/MyDrive/DR/"

# # Open the zip file for reading
# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     # Extract all files to the given directory
#     zip_ref.extractall(extract_path)

In [ ]:
for root, dirs, files in os.walk(dir_path):
    # Print the path of each file
    for file in files:
        print(os.path.join(root, file))

In [ ]:
os.listdir(dir_path)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import image
from PIL import Image,UnidentifiedImageError



In [ ]:
def load_imgs(impath):
    imgs=[]
    label=[]
    l1=os.listdir(impath)
    for i in l1:
        l2=os.listdir(impath+'/'+i)
        for j in l2:
                try:
                  filepath=impath+i+'/'+j
                  img=Image.open(filepath)
                  imgs.append(np.array(img))
                  label.append(i)
                except UnidentifiedImageError as e:
                  print(f'Error reading image at file path {filepath}: {e}')
                  continue
    return np.array(imgs),label




In [ ]:
x,y=load_imgs(dir_path)
print(x.shape)
print(len(y))

In [ ]:
print(y)

In [ ]:
unique_categories, category_counts = np.unique(y, return_counts=True)
for i in range(len(unique_categories)):
    print(f"Category '{unique_categories[i]}' occurs {category_counts[i]} times.")


In [ ]:
#Plot a bar graph
plt.bar(unique_categories, category_counts)
plt.title('Category Counts')
plt.xlabel('Category')
plt.ylabel('Count')
plt.show()

In [ ]:
target=pd.Series(y,dtype='category')
print(target)

In [ ]:
#Encoding the categories:
t=target.cat.codes
print(t)

In [ ]:
# get the unique categories in the Series
unique_categories = target.unique()

# create a dictionary that maps each category to its corresponding integer code
code_map = dict(zip(unique_categories, t.unique()))

print(code_map)

In [ ]:
from sklearn.model_selection import train_test_split
train_x,test_x,train_y,test_y=train_test_split(x,t,test_size=0.2,shuffle=True)

print(train_x.shape)
print(test_x.shape)
print(train_y.shape)
print(test_y.shape)

In [ ]:
# import numpy as np
# from tensorflow.keras import layers
# from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout
# from tensorflow.keras.models import Model, load_model
# from tensorflow.keras.preprocessing import image
# from keras.utils import layer_utils
# # from keras.utils.data_utils import get_file
# # from keras.applications.imagenet_utils import preprocess_input
# # import pydot
# # #from IPython.display import SVG
# # from keras.utils.vis_utils import model_to_dot
# # from tensorflow.keras.utils import plot_model

# from tensorflow.keras.initializers import glorot_uniform
# import scipy.misc
# from matplotlib.pyplot import imshow

# from tensorflow.keras.initializers import glorot_uniform
# from tensorflow.keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# define the augmentation transformations to apply
datagen = ImageDataGenerator(
    rescale=1./255,
    brightness_range=[1 - 20/255, 1 + 20/255],
    zoom_range=0.2,
    rotation_range=180,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,

)

train_gen = datagen.flow(train_x, train_y, batch_size=32)

validation_datagen = ImageDataGenerator(
    rescale=1./255,
    brightness_range=[1 - 20/255, 1 + 20/255],
    zoom_range=0.2,
    rotation_range=180,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,

)
validation_gen=validation_datagen.flow(test_x,test_y,batch_size=32)



In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os
import zipfile
import random
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam

In [ ]:
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16

# Load Pretrained VGG16 Model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add Global Average Pooling Layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(5, activation='softmax')(x)

# Create the model
model = Model(inputs=base_model.input, outputs=predictions)



In [ ]:
for layer in base_model.layers:
    layer.trainable = False


# Compile your model with sigmoid focal cross-entropy loss
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

train_x=train_x/255.0
test_x=test_x/255.0

# Compile Model
history = model.fit(train_x, train_y, epochs=10, batch_size=32, validation_data=(test_x, test_y))



In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# get the predicted class probabilities for the validation set
y_pred_prob = model.predict(validation_gen)

# get the predicted class for each sample in the validation set
y_pred = np.argmax(y_pred_prob, axis=1)

# get a batch of validation images and their true labels
x_val, y_val = next(validation_gen)

# select a random image from the batch
index = np.random.randint(len(x_val))

# get the predicted label for the selected image
label_pred = y_pred[18]

# get the true label for the selected image
label_true = y_val[18]

# get the image array for the selected image
img = x_val[18]

# display the image with the predicted and true labels
plt.imshow(img)
plt.title(f"Predicted label: {label_pred}, True label: {label_true}")
plt.show()